In [2]:
import torch
import yaml
from utils.builder import build_vae_model
from models.condition import ImageEncoder
from models.mar import MAREncoderDecoder
from thsolver.config import parse_args
import sys 
import matplotlib.pyplot as plt
import ocnn
from PIL import Image
import os
import copy
from ognn.octreed import OctreeD
from utils import utils
torch.cuda.set_device(0)

2025-01-18 02:34:54.801528: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-18 02:34:54.803844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-18 02:34:54.852177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-18 02:34:55.562005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# log_path = '/mnt/sdc/wangrh/workspace/OctAR-solver/logs/sketch/airplane_p1024_d8'
log_path = '/mnt/sdc/wangrh/workspace/OctAR-solver/logs/image/car_p1024_d8'

sys.argv = ['']  # Reset sys.argv
sys.argv.extend(['--config', log_path + '/all_configs.yaml'])
flags = parse_args(backup=False)
flags

CfgNode({'BASE': ['configs/shapenet_octar.yaml'], 'SOLVER': CfgNode({'alias': '', 'gpu': (0, 1, 2, 3), 'run': 'generate', 'logdir': 'logs/image/car_p1024_d8', 'ckpt': '', 'ckpt_num': 10, 'type': 'adamw', 'weight_decay': 0.01, 'clip_grad': -1.0, 'max_epoch': 400, 'warmup_epoch': 20, 'warmup_init': 0.001, 'eval_epoch': 1, 'eval_step': -1, 'test_every_epoch': 1, 'log_per_iter': 50, 'best_val': 'min:loss', 'zero_grad_to_none': False, 'use_amp': True, 'lr_type': 'constant', 'lr': 5e-05, 'lr_min': 0.0001, 'gamma': 0.1, 'milestones': (120, 180), 'lr_power': 0.9, 'port': 10001, 'progress_bar': True, 'rand_seed': 0, 'empty_cache': 50, 'expand_ckpt': False, 'step_size': (160, 240), 'resolution': 256, 'save_sdf': False, 'sdf_scale': 0.9}), 'DATA': CfgNode({'train': CfgNode({'name': 'shapenet', 'disable': False, 'pin_memory': True, 'depth': 8, 'full_depth': 3, 'orient_normal': '', 'distort': False, 'scale': 0.0, 'uniform': False, 'jitter': 0.0, 'interval': (1, 1, 1), 'angle': (180, 180, 180), 'fli

In [4]:
device = 'cuda'

model = MAREncoderDecoder(vqvae_config=flags.MODEL.VQVAE, **flags.MODEL.GPT)
vqvae = build_vae_model(flags.MODEL.VQVAE)
sketch_encoder = ImageEncoder(flags.MODEL.GPT.condition_encoder)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
vqvae_checkpoint = torch.load(flags.MODEL.vqvae_ckpt, weights_only=True, map_location="cpu")
vqvae.load_state_dict(vqvae_checkpoint)
print("Load VQVAE from", flags.MODEL.vqvae_ckpt)

Load VQVAE from saved_ckpt/vqvae_huge_im5_bsq32_200epoch.pth


In [6]:
ar_checkpoint = log_path + '/checkpoints/00118.model.pth'
model_checkpoint = torch.load(ar_checkpoint, map_location="cpu")
model.load_state_dict(model_checkpoint)
print("Load MAR from", ar_checkpoint)

Load MAR from /mnt/sdc/wangrh/workspace/OctAR-solver/logs/image/car_p1024_d8/checkpoints/00118.model.pth


In [7]:
model = model.to(device)
vqvae = vqvae.to(device)
sketch_encoder = sketch_encoder.to(device)

In [8]:
depth = flags.DATA.test.depth
full_depth = flags.DATA.test.full_depth
depth_stop = flags.MODEL.depth_stop

In [9]:
def generate_by_sketch(sketch_path, sketchname='default'):
    sketch = Image.open(sketch_path)

    with torch.no_grad():
        condition = sketch_encoder(sketch, device=device)
        octree_out = ocnn.octree.init_octree(
            depth=depth,
            full_depth=full_depth,
            batch_size=1,
            device=device,
        )
        with torch.autocast('cuda', enabled=flags.SOLVER.use_amp):
            octree_out, vq_code = model.generate(
                octree=octree_out,
                depth_low=full_depth,
                depth_high=depth_stop,
                vqvae=vqvae,
                condition=condition,
                cfg_scale=None
            )

    export_path = f'results-inference/{sketchname}'

    index = 'output'

    # Export octrees
    for d in range(full_depth+1, depth_stop+1):
        utils.export_octree(octree_out, d, os.path.join(
            log_path ,export_path), index=f'octree_{d}')

    # Decode the mesh
    for d in range(depth_stop, depth):
        split_zero_d = torch.zeros(
            octree_out.nnum[d], device=octree_out.device).long()
        octree_out.octree_split(split_zero_d, d)
        octree_out.octree_grow(d + 1)
    doctree_out = OctreeD(octree_out)
    with torch.no_grad():
        output = vqvae.decode_code(
            vq_code, depth_stop, doctree_out,
            copy.deepcopy(doctree_out), update_octree=True)

    # extract the mesh
    utils.create_mesh(
        output['neural_mpu'],
        os.path.join(log_path, export_path, f"output.obj"),
        size=flags.SOLVER.resolution,
        level=0.002, clean=True,
        bbmin=-flags.SOLVER.sdf_scale,
        bbmax=flags.SOLVER.sdf_scale,
        mesh_scale=flags.DATA.test.points_scale,
        save_sdf=flags.SOLVER.save_sdf)

    # Save the sketch image
    sketch.save(os.path.join(log_path, export_path, f"input.png"))

In [12]:
# sketch_paths = [f'/mnt/sdc/wangrh/workspace/OctAR-solver/logs/sketch/airplane_p1024_d8/results/images/{i}.png' for i in range(1, 50)]
sketch_path = '/mnt/sdc/weist/data/ShapeNet/fid_images/car/view_0'
sketch_paths = os.listdir(sketch_path)
sketch_paths = [os.path.join(sketch_path, i) for i in sketch_paths]
import random
random.shuffle(sketch_paths)
sketch_paths[:5]

['/mnt/sdc/weist/data/ShapeNet/fid_images/car/view_0/71a1122469df0bb8cc8ad0deed05da5c_0.png',
 '/mnt/sdc/weist/data/ShapeNet/fid_images/car/view_0/555a48617127033c25ad20bb66839d61_0.png',
 '/mnt/sdc/weist/data/ShapeNet/fid_images/car/view_0/6b7f573997bc6f8c953536e7c905b301_0.png',
 '/mnt/sdc/weist/data/ShapeNet/fid_images/car/view_0/42b0bc0fd8e292afbbecf72e56fe7e5d_0.png',
 '/mnt/sdc/weist/data/ShapeNet/fid_images/car/view_0/a07227a41b866d26473f10e6caaeca56_0.png']

In [ ]:
for i, sketch_path in enumerate(sketch_paths[:400]):
    print(f"Generating {i+1}/{len(sketch_paths)}")
    try:
        generate_by_sketch(sketch_path, f"sketch_{i+1}")
    except:
        pass
    print("Done")

Generating 1/5247


  0%|          | 0/64 [00:00<?, ?it/s]/home/wangrh/anaconda3/envs/torch2.3/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1/1 [00:00<00:00,  3.99it/s]


Done
Generating 2/5247


100%|██████████| 1/1 [00:00<00:00,  6.51it/s]


Done
Generating 3/5247


...

100%|██████████| 1/1 [00:00<00:00,  6.94it/s]


Done
Generating 344/5247


 23%|██▎       | 60/256 [00:23<01:15,  2.60it/s]


Done
Generating 345/5247


 89%|████████▉ | 57/64 [00:11<00:01,  4.53it/s]